## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from rasterio import features
from shapely import wkt
import os
from glob import glob
from tqdm.auto import tqdm
from scipy.ndimage import distance_transform_edt
from concurrent.futures import ProcessPoolExecutor
from itertools import product

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = 'Times New Roman'

## Define the Parameters (useful for command line run using papermill)

In [ ]:
CROP = 'ww'
DISTANCE = 2.5
EPSG = 25832

# Path of the directories
WORK_DIR = '/beegfs/halder/GITHUB/RESEARCH/landscape-yield-analysis/'
os.chdir(WORK_DIR)
MAIN_DATA_DIR = '/beegfs/halder/DATA/'
WORK_DATA_DIR = os.path.join(WORK_DIR, 'data')
WORK_TEMP_DIR = os.path.join(WORK_DIR, 'temp')

OUT_DIR = os.path.join(WORK_DIR, 'output', CROP, str(DISTANCE))
if os.path.exists(OUT_DIR) == None:
    os.makedirs(OUT_DIR, exist_ok=True)
    print('Output directory successfully created!')
else:
    print('Output directory already existed.')

## Load Hexagonal Grid for Germany


In [ ]:
# Path to grid shapefile
GRID_PATH = os.path.join(WORK_DATA_DIR, 'VECTOR', f'DE_Hexbins_{DISTANCE}sqkm_EPSG_{EPSG}.shp')

# Load grid as a GeoDataFrame and retain relevant columns
grids_gdf = gpd.read_file(GRID_PATH)
grids_gdf = grids_gdf[['id', 'geometry']]
grids_gdf['id'] = grids_gdf['id'].astype(int)

print('Successfully read the grids!')
grids_gdf.plot();

## Calculate Euclidean Distance from Cropland to Other LULC Features

### Reproject and Resample the LULC Raster

In [ ]:
# Read the Cropland layer
lulc_path = os.path.join(MAIN_DATA_DIR, 'DE_ESA_WORLDCOVER_10M_2021_V200', 'ESA_WorldCover_2021_DE_WGS84.tif')
cropland_path = os.path.join(MAIN_DATA_DIR, 'DE_Crop_Types_2017_2021', f'{CROP}_mask_combined.tif')

# Open source raster
with rio.open(cropland_path) as src:
    cropland = src.read(1)
    cropland_profile = src.profile
    cropland_transform = src.transform
    cropland_crs = src.crs
    cropland_shape = (src.height, src.width)
    
# with rio.open(lulc_path) as src:
#     lulc = src.read(1)
#     lulc_transform = src.transform
#     lulc_crs = src.crs
    
#     # Create an empty array for resampled data
#     lulc_resampled = np.empty(cropland_shape, dtype=lulc.dtype)

#     # Reproject & resample LULC to match cropland grid
#     reproject(
#         source=lulc,
#         destination=lulc_resampled,
#         src_transform=lulc_transform,
#         src_crs=lulc_crs,
#         dst_transform=cropland_transform,
#         dst_crs=cropland_crs,
#         resampling=Resampling.nearest
#     )

# # Save the resampled raster
# cropland_profile.update(
#     dtype=lulc_resampled.dtype,
#     count=1,
#     compress='lzw'
# )

# with rio.open(os.path.join(WORK_TEMP_DIR, 'lulc_resampled.tif'), 'w', **cropland_profile) as dst:
#     dst.write(lulc_resampled, 1)

### Calculate Distance

In [ ]:
# Define LULC codes
lulc_class_code = {
    'tree_cover':10,
    'shrubland': 20,
    'grassland': 30,
    'builtup': 50,
    'bare': 60,
    'water': 80,
    'wetland': 90,
    'mangroves': 95,
    'moss_lichen': 100
}

def dist_to_lulc_per_hexbin(task):
    hex_id, geometry_wkt, lulc, lulc_code = task
    geometry = wkt.loads(geometry_wkt)
    buffer_poly = geometry.buffer(buffer_distance)

    # Open rasters inside worker
    with rio.open(os.path.join(WORK_TEMP_DIR, 'lulc_resampled.tif')) as lulc_src, \
         rio.open(cropland_path) as crop_src:

        # Mask LULC and cropland inside buffer
        lulc_img, lulc_transform = mask(lulc_src, [buffer_poly], crop=True)
        crop_img, _ = mask(crop_src, [buffer_poly], crop=True)

        lulc_data = lulc_img[0]
        cropland_data = crop_img[0]
        cropland_data = np.where(cropland_data>0, 1, 0)

        # If no target LULC class, return NaN
        if not np.any(lulc_data == lulc_code):
            return {'id': hex_id, f'mean_dist_to_{lulc}': np.nan, f'median_dist_to_{lulc}': np.nan}

        # Compute distance transform
        lulc_mask = (lulc_data == lulc_code)
        dist_map = distance_transform_edt(~lulc_mask) * pixel_size

        # Cropland mask in buffer
        cropland_mask_in_buffer = (cropland_data == 1)

        # Rasterize hexbin geometry into buffer extent
        hexbin_mask_in_buffer = features.rasterize(
            [(geometry, 1)],
            out_shape=dist_map.shape,
            transform=lulc_transform,
            fill=0,
            dtype=np.uint8
        ).astype(bool)

        # Cropland pixels inside hexbin (all in buffer extent)
        cropland_in_hexbin_in_buffer = cropland_mask_in_buffer & hexbin_mask_in_buffer

        # Apply mask to dist_map
        dist_cropland_in_hexbin = np.where(cropland_in_hexbin_in_buffer, dist_map, np.nan)

        mean_dist = np.nanmean(dist_cropland_in_hexbin)
        median_dist = np.nanmedian(dist_cropland_in_hexbin)

        return {'id': hex_id, f'mean_dist_to_{lulc}': mean_dist, f'median_dist_to_{lulc}': median_dist}

buffer_distance = (DISTANCE * 1000) // 2
pixel_size = 10

for lulc, lulc_code in lulc_class_code.items():
    print('*' * 20 + f' {lulc} ' + '*' * 20)
    # Prepare list of tasks: hex_id and geometry WKT
    tasks = [(row['id'], row.geometry.wkt, lulc, lulc_code) for idx, row in grids_gdf.iterrows()]

    # Parallel execution
    results = []
    with ProcessPoolExecutor(max_workers=50) as executor:
        for res in tqdm(executor.map(dist_to_lulc_per_hexbin, tasks), total=len(tasks), desc='Processing hexbins'):
            results.append(res)

    results = pd.DataFrame(results)
    results.to_csv(os.path.join(WORK_TEMP_DIR, f'distance_to_{lulc}.csv'), index=False)